# A simple chatbot agent

## Prepare the environment

In [ ]:
import sys
sys.path.append('../../..')

from autogen import ConversableAgent, register_function

# from inswitch.llm.model import get_openai_model_config
from inswitch.agent.basic import get_chat_agent, get_fixed_reply_agent
from inswitch.util import second_last_msg

from rdflib import Graph, URIRef

## The functions to query the complete kg
TODO: generalize it to work on given ttl files.

In [ ]:
kg = Graph()
kg.parse("../data/anonimized_data_fill.ttl")

def query_kg(query:str)->str:
    skg = kg.query(query)

    result = ""
    for s, p, o in skg:
        # Use namespace_manager to abbreviate URIs
        subject_prefix = kg.namespace_manager.qname(s)
        predicate_prefix = kg.namespace_manager.qname(p)
        object_prefix = kg.namespace_manager.qname(o) if isinstance(o, URIRef) else f'"{o}"'
        
        # Print in the desired format
        result = result + f"{subject_prefix} {predicate_prefix} {object_prefix}\n"
    return result

def get_sample_kg()->str:
    return query_kg("""
        SELECT (SAMPLE(?subject) AS ?sampleSubject) ?predicate  (SAMPLE(?object) AS ?sampleObject)
        WHERE {
        ?subject ?predicate ?object .
        }
        GROUP BY ?predicate
    """)

def get_namespaces()->str:

    return """
@prefix fill: <https://intendproject.eu/fill/>.
@prefix intend: <https://intendproject.eu/schema/>.
@prefix schema: <https://schema.org/>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
    """

    # TODO: This does not work... Need to fix
    # return "\n".join(
    #     f"{prefix}: {namespace}" 
    #     for prefix, namespace in kg.namespace_manager.namespaces()
    #     if prefix in kg.namespace_manager.store.prefixes()
    # )


# print(get_sample_kg())
# print(get_namespaces())

## Here are the API invocation agents

In [ ]:
rdf_tool_driver_system_message = '''
You are a helpful assistant. From a user intent (provided in the context), you will
try to get all the knowledges that may be relevant to this intent.
You have access to a Knowledge Graph, and you can generate and execute sparql queries
to obtain a subgraph from the knowledge graph, and that subgraph is all the knowledge
that might be relevant to the intent.
You may do it step by step, starting from listing all the service descriptions, 
and see which exact service the intent is about. From the intent, you
can also know which active machine the user owns. Then, you can 
extract all the triples methods either the service or the active machine.
Do not try to search keywords in service descriptions, but compare the description with the intent to decide
which one is relevant.
The SPARQL queries you generate must always return full tripes in the form of subject predict object.
Do not forget the predict!
'''

rdf_tool_driver = get_chat_agent(
    "rdf_tool_driver",
    system_message = rdf_tool_driver_system_message
)

rdf_tool_executor = get_fixed_reply_agent(
    "rdf_tool_executor",
    reply = ""
)

register_function(
    query_kg,
    caller = rdf_tool_driver,
    executor = rdf_tool_executor,
    description = "This is the function to query the knowledge graph. The input is a rdf query, "
        "and the output is a list of triples as the query result."
        
)

rdf_tool_executor.register_nested_chats(
    [
        {
            "recipient": rdf_tool_driver,
            "max_turns": 5,
            "summary_method": second_last_msg
        }
    ],
    trigger = lambda sender: sender not in [rdf_tool_driver]
)



## Here we go!
A simple sequence: user_intent -> api_doc_provider -> nerv_tool_executor

Current intent is hard coded.

In [ ]:
user_intent = '''I am Customer1 and own Active_machine6. 
I want to add a new service to my machine to record all the alarms
'''

intent_provider = get_fixed_reply_agent(
    'intent_provier',
    reply = user_intent
)

moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)

decision_chat_results = moderator.initiate_chats(
    [
        {
            "recipient": intent_provider,
            "message": "what do you want?",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": rdf_tool_executor,
            "message": f"Sample triples in the kg: {get_sample_kg()}. Namespaces: {get_namespaces()}",
            "max_turns": 1,
            "summary_method": "last_msg"
        }
    ]
)



## Next steps:

1. Replace current api_doc_provider (a fixed_reply_agent) by a RAG assistant, that extracts from a complete API document the parts that are relevant to DNA and deployment.
2. Add in the beginning of the sequence another agent (or agents) to extract the list of workloads. Eventually, the input intent should be like "I want to be able to monitor the thermal stability of machine M0001"